In [ ]:
!pip install pyaudio numpy scipy matplotlib

In [ ]:
import pyaudio
import numpy as np
import tkinter as tk

# Audio Configuration
CHUNK = 1024           # Number of audio samples per frame
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100           # Sampling rate (Hz)

# Initialize PyAudio
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True,
                frames_per_buffer=CHUNK)

def find_frequency_and_amplitude(data, rate):
    """
    Given a buffer of audio data and sample rate, find the dominant frequency and amplitude.
    Returns: freq (float), amplitude (float), fft_magnitude (1D array)
    """
    # Convert raw data to numpy array
    samples = np.frombuffer(data, dtype=np.int16).astype(np.float32)
    print(samples)
    
    # Apply a windowing function (Hanning window)
    windowed = samples * np.hanning(len(samples))
    
    # Compute FFT using rfft (real-valued fft)
    fft_spectrum = np.fft.rfft(windowed)
    fft_magnitude = np.abs(fft_spectrum)
    
    # Find the peak frequency bin
    peak_bin = np.argmax(fft_magnitude)
    
    # Convert bin index to frequency
    freq = peak_bin * (rate / CHUNK)
    
    # Amplitude at the peak frequency
    amplitude = fft_magnitude[peak_bin]
    
    return freq, amplitude, fft_magnitude

# Set up Tkinter window
root = tk.Tk()
root.title("Real-Time Audio Frequency and Amplitude")

# Labels for frequency and amplitude
freq_label = tk.Label(root, text="Frequency: -- Hz", font=('Helvetica', 14))
freq_label.pack(pady=5)

amp_label = tk.Label(root, text="Amplitude: --", font=('Helvetica', 14))
amp_label.pack(pady=5)

# Canvas for plotting the spectrum
canvas_width = 600
canvas_height = 200
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="white")
canvas.pack(pady=10)

# We will dynamically adjust the vertical scale to fit the data
max_amplitude = 1.0  # start with some default max amplitude

def update_audio_info():
    global max_amplitude
    # Read audio chunk
    data = stream.read(CHUNK, exception_on_overflow=False)
    
    # Compute frequency, amplitude, and full FFT
    freq, amplitude, fft_magnitude = find_frequency_and_amplitude(data, RATE)
    
    # Update labels
    freq_label.config(text=f"Frequency: {freq:.2f} Hz")
    amp_label.config(text=f"Amplitude: {amplitude:.2f}")
    
    # Clear previous drawing
    canvas.delete("all")
    
    # Update max amplitude if needed (for scaling)
    if amplitude > max_amplitude:
        max_amplitude = amplitude
    
    # Frequencies for the rfft result
    freqs = np.fft.rfftfreq(CHUNK, 1.0/RATE)
    
    # Draw the spectrum as a line
    # Map frequency -> x coordinate, amplitude -> y coordinate
    # x: 0 Hz -> x=0, max freq -> x=canvas_width
    # y: amplitude scaled so that max_amplitude maps to near top of the canvas
    # We'll invert y so larger amplitude is higher on the canvas
    max_freq = RATE / 2.0  # Nyquist frequency
    points = []
    for i, mag in enumerate(fft_magnitude):
        x = (freqs[i] / max_freq) * canvas_width
        y = canvas_height - (mag / max_amplitude) * (canvas_height - 10)
        points.append((x,y))
    
    # Create a polyline from these points
    # We can simply connect them with a line
    for i in range(len(points)-1):
        canvas.create_line(points[i][0], points[i][1],
                           points[i+1][0], points[i+1][1],
                           fill="blue", width=2)
    
    # Draw a vertical red line at the dominant frequency
    peak_x = (freq / max_freq) * canvas_width
    canvas.create_line(peak_x, canvas_height, peak_x, 0, fill="red", dash=(4,2))
    
    # Schedule the next update
    root.after(50, update_audio_info)  # update every 50 ms (adjust as needed)

# Start the update loop
update_audio_info()
print('hi')

def on_closing():
    # Stop audio stream and close PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)

root.mainloop()


OSError: [Errno -9988] Stream closed